In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)  # отобразить все колонки
pd.set_option('display.max_rows', None)

In [6]:
df = pd.read_csv('../data/_data.with_new_cols.csv')

/tmp/ipykernel_67710/2416377651.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/_data.with_new_cols.csv')


In [7]:
df[['floor', 'n_of_floors', 'build_type']] = df['Дом'].str.extract(r'(\d+)[/\\](\d+),?\s*([А-Яа-яA-Za-z\s]+)?')  # Добавила 3 колонки

In [8]:
df = df.drop_duplicates(
    subset=['Адрес', 'Общая площадь', 'rooms_qty', 'floor', 'n_of_floors', 'Ремонт'],
    keep='first'
).copy() # Убрали дублирующие строки 454


In [9]:
# Ремонт, Можно с детьми/животными, Название ЖК чистое, Парковка, Station, Line, AdmArea, District - заполнение NaN строкой "Данные отсуствуют"

df["Ремонт"] = df["Ремонт"].fillna("Данные отсутствуют")
df["Можно с детьми/животными"] = df["Можно с детьми/животными"].fillna("Данные отсутствуют")
df["Название ЖК чистое"] = df["Название ЖК чистое"].fillna("Данные отсутствуют")
df["Парковка"] = df["Парковка"].fillna("Данные отсутствуют")
df["Station"] = df["Station"].fillna("Данные отсутствуют")
df["Line"] = df["Line"].fillna("Данные отсутствуют")
df["AdmArea"] = df["AdmArea"].fillna("Данные отсутствуют")
df["District"] = df["District"].fillna("Данные отсутствуют")

In [10]:
# Балкон, Окна, Санузел, Телефоны, Описание - заполнение NaN, None значением False(0), значением True(1) - заполненные значения

df["Балкон"] = df["Балкон"].notna().astype(int)
df["Окна"] = df["Окна"].notna().astype(int)
df["Санузел"] = df["Санузел"].notna().astype(int)
df["Телефоны"] = df["Телефоны"].notna().astype(int)
df["Описание"] = df["Описание"].notna().astype(int)

In [11]:
# Создаем новые колонки "Мебель в комнатax" и "Мебель на кухне" из колонки "Дополнительно" - заполняем значениями True/False (0, 1)

df["Мебель в комнатах"] = df["Дополнительно"].str.contains("Мебель в комнатах", na=False).astype(int)
df["Мебель на кухне"] = df["Дополнительно"].str.contains("Мебель на кухне", na=False).astype(int)

In [12]:
df.fillna({'Высота потолков, м': df['Высота потолков, м'].median()}, inplace=True) # Заполнили медианой пропуски

In [13]:
df['Лифт'] = (
    df['Лифт']
    .astype(str)          
    .str.lower().str.strip()
    .apply(lambda x: 0 if x == 'nan' else 1)
)  # Значения в колонке Лифт заменены на 1 - есть значение / - nan

In [14]:
df['Мусоропровод'] = df['Мусоропровод'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'
df['room_type'] = df['room_type'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'
df['build_type'] = df['build_type'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'

In [15]:
df['Город'] = df['Город'].apply(lambda x: 1 if x == 'Москва' else 0) # Закодировали 'Город' 1/0

In [16]:
df[['street', 'bld.']] = df['Адрес'].str.split(',',expand=True).iloc[:, 1:3] # Выделили адрес в две колонки streen, bld.

In [17]:
# Оставляем в датафрейме только предложения со стоимостью, указанной в рублях
df = df[df['Цена'].str.contains("руб", case=False, na=True)]

In [18]:
# Разделение параметра 'Путь до метро' на 2 - время (int) и досягаемость (пешком/на машине)
df[['Время до метро', 'Досягаемость']] = df['Путь до метро'].str.extract(r'(\d+)\s*мин\s*(.*)')
df['Время до метро'] = pd.to_numeric(df['Время до метро'], errors='coerce').astype('Int64')

In [19]:
#df['Время до метро'] = df['Время до метро'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'

In [20]:
df = df.drop(['Unnamed: 0','Количество комнат', 'Дом', 'Название ЖК', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Цена', 'Площадь комнат, м2',
              'Дополнительно', 'Серия дома', 'Ссылка на объявление', 'ЖК год', 'Станция', 'Путь до метро'], axis=1)

In [21]:
missing_counts = df.isna().sum()
unique_counts = df.nunique(dropna=False)
df_types = df.dtypes
summary = pd.DataFrame({
    'Пропуски (шт.)': missing_counts,
    'Уникальных значений (вкл. NaN)': unique_counts,
    'Типы данных': df_types
})
summary

,Пропуски (шт.),Уникальных значений (вкл. NaN),Типы данных
ID объявления,0,19261,int64
Парковка,0,6,object
Телефоны,0,1,int64
Описание,0,1,int64
Ремонт,0,5,object
Балкон,0,2,int64
Окна,0,2,int64
Санузел,0,2,int64
Можно с детьми/животными,0,4,object
"Высота потолков, м",0,88,float64


In [22]:
df.fillna({'Время до метро': df['Время до метро'].median()}, inplace=True) # Заполнили медианой пропуски

In [23]:
df['Досягаемость'] = df['Досягаемость'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'

In [24]:
# Для нормализации данных по колонкам Цена за месяц и Общая площадь применено Логарифмирование (log-transform)
# Тк это особенно полезно для MAPE, уменьшает разброс относительных ошибок, делает MAPE более устойчивым

df["Цена за месяц"] = np.log1p(df["Цена за месяц"]) # обратное после предсказания np.expm1(df["Цена за месяц"])
df["Общая площадь"] = np.log1p(df["Общая площадь"])

In [25]:
def convert_to_meters(value):
    try:
        num = float(value)
        # Если число меньше 10 (предполагаем, что это метры с дробной частью)
        # Или если число больше 100 (предполагаем, что это сантиметры)
        if 10 < num < 100:  # Значения больше 100 считаем сантиметрами
            return num / 10
        elif num > 100:
            return num / 100
        else:  # Остальные считаем метрами
            return num
    except:
        return np.nan  # Для некорректных значений

In [26]:
# Применяем функцию convert_to_meters
df['Высота корректная, м'] = df['Высота потолков, м'].apply(convert_to_meters)

In [27]:
# Оставляем в датафрейме только предложения с высотой потолка >= 2.4 м
df = df[df['Высота корректная, м'] >= 2.4]